In [4]:
class Point:
    def __init__(self, x, y, degree):
        self.x = x
        self.y = y
        self.degree = degree
        
    def __hash__(self):
        return hash((self.x, self.y, self.degree))
    
    def __eq__(self, other):
        return (self.x, self.y, self.degree) == (other.x, other.y, other.degree)
    
    def __ne__(self, other):
        return not(self == other)
    
    def distance(self, other):
        return mt.sqrt((self.x - other.x) ** 2 + (self.y - other.y) ** 2)
    
    def dump(self):
        print("({}; {}) with degree {}".format(self.x, self.y, self.degree))
        
    def __str__(self):
        return "(" + str(self.x) + "; " + str(self.y) + ") " + str(self.degree)
    
    def __repr__(self):
        return self.__str__()
    
    def angle(self, other):
        if cos_value > 1:
            cos_value = 1
        return mt.acos(cos_value)
   
    def norm(self):
        return mt.sqrt(self.x ** 2 + self.y ** 2)
    
    def eq(self, other):
        return (self.x, self.y) == (other.x, other.y)
    
    
class Skeleton:
    dist = 0
    angle = 0
    
    def __init__(self, features):
        self._threshold_angle = mt.pi / 4
        self._graph = self._build_graph(features)
        self._special_points = self._get_special_points()
        self._compute_features()
        
    def _build_graph(self, features):
        graph = {}
        _, _, _, _, deg = get_edges_deg_rad(features)
        
        max_x = 0
        max_y = 0
        for i in range(0, len(deg)):
            p1 = Point(*deg[i])
            if p1.x > max_x:
                max_x = p1.x
            if p1.y > max_y:
                max_y = p1.y

        for i in range(0, len(deg), 2):
            p1 = Point(*deg[i])
            p2 = Point(*deg[i+1])
            p1.x /= max_x
            p1.y /= max_y
            p2.x /= max_x
            p2.y /= max_y
            if p1 not in graph:
                graph[p1] = set()
            graph[p1].add(p2)
            if p2 not in graph:
                graph[p2] = set()
            graph[p2].add(p1)
                    
        return graph

    def _validate_graph(self, graph):
        for key, value in graph.items():
            if key.degree != len(value):
                key.dump()
                return False
        return True
    
    def _compute_features(self):
        self._compute_relative_len()
        self._compute_directions()
        self._compute_io_directions()
        self._compute_curvature()  
        self._compute_number_of_special_points()
        self._compute_number_of_points()
        self._compute_new_curvature()
        self._compute_average_dist()
        self._compute_angles(list(self._graph.keys())[0])
        self._compute_avg_loop()
        self._compute_degree()
        self._compute_loop_deviation()
        
    def _compute_number_of_special_points(self):
        self.number_of_special_points = len(self._get_special_points())
        
    def _get_sum_of_deviations(self, start, visited = None):
        if visited is None:
            visited = set()
        visited.add(start)
        for neighbour1 in self._graph[start]:
            for neighbour2 in self._graph[start]:
                if neighbour2 != neighbour1:
                    if neighbour1.distance(neighbour2) > 0:
                        self.new_curvature += ((start.x - (neighbour1.x + neighbour2.x) / 2) ** 2 + \
                                                (start.y - (neighbour1.y + neighbour2.y) / 2) ** 2) / \
                                                (neighbour1.distance(neighbour2))
        for next in self._graph[start] - visited:
            self._get_sum_of_deviations(next, visited)
        return visited
    
    def _compute_new_curvature(self):
        self.new_curvature = 0
        self._get_sum_of_deviations(list(self._graph.keys())[0])
        self.new_curvature /= self.number_of_points
        
    def _compute_average_dist(self):
        self.average_dist = self.dist / self.number_of_points
        
    def _compute_relative_len(self):
        self._get_graph_len()
        if len(self._special_points) < 2:
            self.relative_dist = [0, 0, 0]
            return
        
        dist = []
        for i in range(len(self._special_points)-1):
            d = 0
            path = list(self._bfs_paths(self._special_points[i], self._special_points[i+1]))
            if not len(path):
                continue
            path = path[0]
            for k in range(len(path)-1):
                d += path[k].distance(path[k+1])
            dist.append(d / self.dist)
        
        if len(dist) == 0:
            self.relative_dist = [0, 0, 0]
            return
        
        arr = np.array(dist)
        self.relative_dist = [np.max(arr), np.min(arr), np.mean(arr)]
       
    def _get_graph_len(self):
        self.dist = 0
        i = 0
        while self.dist == 0:
            self._dfs_full_len(list(self._graph.keys())[i])
            i += 1
        
    def _compute_directions(self):
        dirx = []
        diry = []
        dist_arr = []
        dir = []
        if len(self._special_points) < 2:
            self.directions_f = [0, 0, 0, 0, 0, 0, 0, 0, 0]
            return
        
        for i in range(len(self._special_points)-1):
            dist = self._special_points[i].distance(self._special_points[i+1])
            dx = self._special_points[i+1].x - self._special_points[i].x
            dy = self._special_points[i+1].y - self._special_points[i].y
            dirx.append(dx)
            diry.append(dy)
            dist_arr.append(dist)
            dir.append((dx, dy, dist))
        
        self._directions = dir
        self.directions_f = [np.min(np.array(dirx)), np.max(np.array(dirx)), 
                             np.mean(np.array(dirx)), np.min(np.array(diry)), 
                             np.max(np.array(diry)), np.mean(np.array(diry)), 
                             np.min(np.array(dist_arr)), np.max(np.array(dist_arr)),
                             np.mean(np.array(dist_arr))]
        

    def _compute_io_directions(self):
        odirx = []
        odiry = []
        odirdist = []
        idirx = []
        idiry = []
        idirdist = []
        if len(self._special_points) < 2:
            self.io_directions = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
            return
        
        for i in range(len(self._special_points)):
            for v in self._graph[self._special_points[i]]:
                dist = v.distance(self._special_points[i])
                dx = v.x - self._special_points[i].x
                dy = v.y - self._special_points[i].y
                odirx.append(dx)
                odiry.append(dy)
                odirdist.append(dist)
            for key, value in self._graph.items():
                if self._special_points[i] in value:
                    dist = key.distance(self._special_points[i])
                    dx = key.x - self._special_points[i].x
                    dy = key.y - self._special_points[i].y
                    idirx.append(dx)
                    idiry.append(dy)
                    idirdist.append(dist)
        self.io_directions = [np.min(np.array(idirx)), np.max(np.array(idirx)), 
                              np.mean(np.array(idirx)), np.min(np.array(odirx)), 
                              np.max(np.array(odirx)), np.mean(np.array(odirx)), 
                              np.min(np.array(idiry)), np.max(np.array(idiry)), 
                              np.mean(np.array(idiry)), np.min(np.array(odiry)), 
                              np.max(np.array(odiry)), np.mean(np.array(odiry)), 
                              np.min(np.array(idirdist)), np.max(np.array(idirdist)), 
                              np.mean(np.array(idirdist)), np.min(np.array(odirdist)), 
                              np.max(np.array(odirdist)), np.mean(np.array(odirdist))]
        
    def _compute_number_of_points(self):
        self.number_of_points = len(list(self._graph.keys()))
    
    def _compute_curvature(self):
        curvature = []
        if len(self._special_points) < 2:
            self.curvature = [0, 0, 0]
            return
        
        for i in range(len(self._special_points)-1):
            d = 0
            path = list(self._bfs_paths(self._special_points[i], self._special_points[i+1]))
            dx, dy, dist = self._directions[i]
            for p in path:
                max_dist = 0
                for k in range(len(p)):
                    d = mt.fabs(dy * p[k].x - dx * p[k].y + self._special_points[i].y * self._special_points[i+1].x - self._special_points[i+1].y * self._special_points[i].x)
                    d /= dist
                    if d > max_dist:
                        max_dist = d
                curvature.append(max_dist / dist)
        if len(curvature) == 0:
            self.curvature = [0, 0, 0]
            return
        self.curvature = [np.min(np.array(curvature)), np.max(np.array(curvature)), np.mean(np.array(curvature))]
                    
    def _dfs_full_len(self, start, visited=None):
        if visited is None:
            visited = set()
        visited.add(start)
        for next in self._graph[start] - visited:
            self._dfs_full_len(next, visited)
            self.dist += next.distance(start)
        return visited
    
    def _bfs_paths(self, start, goal):
        queue = [(start, [start])]
        while queue:
            (vertex, path) = queue.pop(0)
            for next in self._graph[vertex] - set(path):
                if next == goal:
                    yield path + [next]
                else:
                    queue.append((next, path + [next]))
    
    def _get_special_points(self):
        points = []
        for key, value in self._graph.items():
            if key.degree == 1 or key.degree == 3:
                if len(value) > 0:
                    points.append(key)
                
        points.sort(key=lambda x: x.x, reverse=True)
        return points
    
    def _compute_angles(self, start, visited=None):
        if visited is None:
            visited = set()
        visited.add(start)
        for next in self._graph[start] - visited:
            v = visited
            v.add(start)
            v.add(next)
            prev = list(self._graph[next] - v)
            if len(prev) > 0:
                prev = prev[0]
            else:
                continue

            if prev.degree == 1:
                continue
            if not prev.eq(next) and not start.eq(next):
                cos_value = mt.fabs(((start.x - next.x) * (next.x - prev.x) + 
                        (start.y - next.y) * (next.y - prev.y)) /
                        (mt.sqrt((next.x - start.x)**2 + (next.y - start.y)**2) *
                         mt.sqrt((prev.x - next.x)**2 + (prev.y - next.y)**2)))
            else:
                cos_value = 1
            self._compute_angles(next, visited)
            if cos_value > 1:
                cos_value = 1
            angle = mt.acos(cos_value)
            if angle > self._threshold_angle:
                self.angle += angle
                
    def _paths(self, start, goal):
        queue = [(start, [])]
        answer = []
        while queue:
            (vertex, path) = queue.pop(0)
            for next in self._graph[vertex] - set(path):
                if next == goal:
                    answer.append(path + [next])
                else:
                    queue.append((next, path + [next]))
        return answer
                
    def _compute_avg_loop(self):
        special_points = self._get_special_points()
        a = []
        s = 0
        for point in special_points:
            a = self._paths(point, point)
        for i in range(len(a)):
            for j in range(len(a[i])):
                if j == len(a[i]) - 1:
                    s += a[i][j].distance(a[i][0])
                else:
                    s += a[i][j].distance(a[i][j + 1])
        if len(a) == 0:
            self.avg_loop = 0
        else:
            self.avg_loop = s / len(a)
    
    def _compute_degree(self):
        special_points = self._get_special_points()
        self.degree_1 = 0
        self.degree_2 = 0
        self.degree_3 = 0
        self.degree_4 = 0
        for point in special_points:
            if point.degree == 1:
                self.degree_1 += 1
            if point.degree == 2:
                self.degree_2 += 1
            if point.degree == 3:
                self.degree_3 += 1
            if point.degree == 4:
                self.degree_4 += 1
                
    def _compute_loop_deviation(self):
        self.loop_deviation = 0
        paths = []
        for point in self._special_points:
            paths = self._paths(point, point)
        if len(paths) == 0:
            return
        for path in paths:
            for p in path:
                self.loop_deviation += p.y - 0.5